# 哈希表_Hashtable

数据依旧保存在数组中，但是通过哈希函数计算的下标访问（注意取模过程）。查找、访问、删除元素的时间复杂度均为O（1）。

### 冲突及解决策略
1.链接法：冲突时将对应的槽编程链式机构。
2.开放寻址法：当槽被占用的时候，重新机选下一个可用的槽。
- 线性探查（linear probing）：当一个槽被占用，找到下一个可用的槽。 
$ h(k, i) = (h^\prime(k) + i) % m, i = 0,1,...,m-1 $
- 二次探查（quadratic probing）：当一个槽被占用，以二次方作为偏移量。
$ h(k, i) = (h^\prime(k) + c_1 + c_2i^2) % m , i=0,1,...,m-1 $
- 双重散列（double hashing）：重新计算hash结果。$ h(k,i) = (h_1(k) + ih_2(k)) % m $

3.Cpython的解决哈希冲突的方式是：j = ((5*j) + 1) mod 2**i

### 扩容
#### 装载因子
已经使用的槽数除以哈希表的大小，通常当负载因子开始超过0.8的时候，就要新开辟空间并且重新进行散列。

#### 重哈希
重新开辟一块新的空间，把原来的哈希表里面的不为空槽的数重新写入到新的哈希表里面（迭代遍历原来的哈希表），插入的方式和哈希表之前的插入方式一样。

python3.3 的扩容策略是扩大为已经使用的槽的数目的两倍。

### 关键的理解点
1.数组保存的对象以及slot的不同的值。

2.通过hash计算数组的下标。

3.在数组中查找key。

4.在数组中查找可供key插入的位置。

## 哈希表的实现(hashtable.py)

In [ ]:
class Array(object):
    """基于实现一个数组的数据结构"""
    def __init__(self, size=32, init=None):  # 加入每个槽位的初始值，还是默认为None
        self._size = size
        self._items = [init] * size

    def __getitem__(self, index):  # Called to implement evaluation of self[index]实现下标访问.
        return self._items[index]

    def __setitem__(self, index, value):  # Called to implement assignment to self[index].
        self._items[index] = value

    def __len__(self):
        return self._size

    def clear(self, value=None):
        for i in range(len(self._items)):
            self._items[i] = value

    def __iter__(self):
        for item in self._items:
            yield item


class Slot(object):
    """定义一个 hash 表 数组的槽
        注意，一个槽有三种状态或者说三种值。相比链接法解决冲突，二次探查法删除一个 key 的操作稍微复杂。
        1.从未使用过，值为HashMap.UNUSED（None）。此槽没有被使用和冲突过，查找时只要找到 UNUSED 就不用再继续探查了
        2.使用过但是 remove 了，此时值为 HashMap.EMPTY，该探查点后边的元素仍可能是有key
        3.槽正在使用 Slot 节点
    """

    def __init__(self, key, value):
        self.key, self.value = key, value


class HashTable(object):
    """从未使用过是UNUSED，使用过但是remove了就是空"""
    UNUSED = None
    EMPTY = Slot(None, None)  # 初始化slot的键值就是None值

    def __init__(self):
        '''初始化表从未使用过，表长为8位'''
        self._table = Array(8, init=HashTable.UNUSED)  # key-value对以slot对象的形式保存在数组中，slot初始值为HashMap.UNUSED
        self.length = 0

    # 将方法转成属性
    @property
    def _load_factor(self):
        # 负载因子超过0.8重新分配空间
        return self.length / float(len(self._table))

    def __len__(self):
        '''取长度'''
        return self.length

    def _hash(self, key):
        '''hash方法，返回绝对值'''
        return abs(hash(key)) % len(self._table)

    def _find_key(self, key):
        """查找key，返回key在数组中的位置
            若index位置的值为UNUSED，说明槽未使用过，key在数组中不存在
            若为EMPTY或key值，则设法返回key在数组中的位置
        """
        index = self._hash(key)
        _len = len(self._table)
        while self._table[index] is not HashTable.UNUSED:
            if self._table[index] is HashTable.EMPTY:  # 若值为EMPTY，继续探查点后边的元素
                index = (index * 5 + 1) % _len
                continue  # 跳过当前循环的剩余语句
            elif self._table[index].key == key:  # 若值不为EMPTY，检查slot的key值是否等于key
                return index
            else:  # 有值且不为key，则继续探查点后边的元素
                index = (index * 5 + 1) % _len
        return None

    def _find_slot_for_insert(self, key):
        '''寻找插槽'''
        index = self._hash(key)
        _len = len(self._table)
        while not self._slot_can_insert(index):
            index = (index * 5 + 1) % _len
        return index

    def _slot_can_insert(self, index):
        '''能够插入数据的插槽'''
        return self._table[index] is HashTable.EMPTY or self._table[index] is HashTable.UNUSED

    def __contains__(self, key):  # 实现散列表的 in 操作
        '''判断是否包含对应的键'''
        index = self._find_key(key)
        return index is not None

    def add(self, key, value):
        """向散列表中添加key-value对
        首先查找key是否已经在散列表中，若已存在，重置其value并返回False；
        若不存在，查找可供插入的槽并插入Slot，并检查负载因子，若有必要则进行重哈希
        """
        index = self._find_key(key)
        if index is not None:  # 更新已存在的key
            self._table[index].value = value
            return False
        else:
            index = self._find_slot_for_insert(key)
            self._table[index] = Slot(key, value)  # 用key-value对构造Slot并插入到数组
            self.length += 1
            if self._load_factor >= 0.8:
                self._rehash()
            return True

    def _rehash(self):
        '''重哈希'''
        oldtable = self._table
        newsize = len(self._table) * 2
        self._table = Array(newsize, HashTable.UNUSED)  # 使用新的大小构造数组
        self.length = 0

        for slot in oldtable:  # 迭代旧表并将非空节点添加到新数组
            if slot is not HashTable.UNUSED and slot is not HashTable.EMPTY:
                index = self._find_slot_for_insert(slot.key)
                self._table[index] = slot
                self.length += 1

    def get(self, key, default=None):
        '''获取哈希表的键值对'''
        index = self._find_key(key)
        if index is None:  # 首先检查key是否存在
            return default
        else:
            return self._table[index].value

    def remove(self, key):
        '''去除哈希表的键值对'''
        index = self._find_key(key)
        if index is None:  # 首先检查key是否存在
            raise KeyError()
        else:
            value = self._table[index].value
            self.length -= 1
            self._table[index] = HashTable.EMPTY
            return value

    def __iter__(self):
        """迭代数组中的slot，若slot不为空，则返回其key值
        :return:
        """
        for slot in self._table:
            if slot not in (HashTable.EMPTY, HashTable.UNUSED):
                yield slot.key


def test_hash_table():
    # 初始化：实例化哈希表
    h = HashTable()
    h.add('a', 0)
    h.add('b', 1)
    h.add('c', 2)
    assert len(h) == 3
    assert h.get('a') == 0
    assert h.get('b') == 1
    assert h.get('hehe') is None

    h.add('a', 3)
    assert h.get('a') == 3

    h.remove('a')
    assert h.get('a') is None
    assert sorted(list(h)) == ['b', 'c']

    n = 50
    for i in range(n):
        h.add(i, i)

    for i in range(n):
        assert h.get(i) == i